In [1]:
with open('input.txt','r',encoding='utf-8') as f:
    text=f.read()

In [6]:
len(text)

1115394

In [7]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [8]:
chars=sorted(list(set(text)))
vocab_size=len(chars)
itos={i:ch for i,ch in enumerate(chars)}
stoi={ch:i for i,ch in itos.items()}
encode=lambda s:[stoi[c] for c in s]
decode=lambda l:''.join(itos[i] for i in l)
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [1]:
import torch
data=encode(text)
data=torch.tensor(data,dtype=torch.long)
print(data[:100])

NameError: name 'encode' is not defined

In [10]:
n=int(0.9*len(data))
train=data[:n]
test=data[n:]
print(train.shape,test.shape)


torch.Size([1003854]) torch.Size([111540])


In [11]:
torch.randint(10,(3,))

tensor([5, 7, 2])

In [12]:
torch.manual_seed(1337)
batch_size=4
block_size=8

def get_batch(split):
    data=train if split=="train" else test 
    ix=torch.randint(len(data)-block_size,(batch_size,))
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1]for i in ix])
    return x,y
xb,yb=get_batch(train)

In [18]:
xb

tensor([[ 6,  1, 52, 53, 58,  1, 58, 47],
        [ 6,  1, 54, 50, 39, 52, 58, 43],
        [ 1, 58, 46, 47, 57,  1, 50, 47],
        [ 0, 32, 46, 43, 56, 43,  1, 42]])

In [19]:
yb

tensor([[ 1, 52, 53, 58,  1, 58, 47, 50],
        [ 1, 54, 50, 39, 52, 58, 43, 58],
        [58, 46, 47, 57,  1, 50, 47, 60],
        [32, 46, 43, 56, 43,  1, 42, 53]])

In [13]:
import torch
import torch.nn as nn 
from torch.nn import functional as F 
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)
    def forward(self,idx,targets=None):
        logits=self.token_embedding_table(idx)
        if targets is None:
            loss=None 
        else:
            B,T,C=logits.shape
            logits=logits.view(B*T,C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits,targets)

        return logits,loss 
    
    def generate(self,idx,max_new_token):
        for _ in range(max_new_token):
            logits,loss=self(idx)
            logits=logits[:,-1,:]
            probs=F.softmax(logits,dim=-1)
            idx.next=torch.multinomial(probs,num_samples=1)
            idx=torch.cat((idx,idx.next),dim=1)
        return idx
    
m=BigramLanguageModel(vocab_size)
logits,loss=m(xb,yb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_token=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.4913, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [20]:
optimizer=torch.optim.AdamW(m.parameters(),lr=1e-3)

In [21]:
batch_size=32
for steps in range(1000):
    xb,yb=get_batch("train")

    logits,loss=m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

3.806382656097412


In [14]:
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_token=500)[0].tolist()))


pdcbf?pGXepydZJSrF$Jrqt!:wwWSzPNxbjPiD&Q!a;yNt$Kr$o-gC$WSjJqfBKBySKtSKpwNNfyl&w:q-jluBatD$Lj;?yzyUca!UQ!vrpxZQgC-hlkq,ptKqHoiX-jjeLJ &slERj KUsBOL!mpJO!zLg'wNfqHAMgq'hZCWhu.W.IBcP 
RFJ&DEs,nw?pxE?xjNHHVxJ&D&vWWToiERJFuszPyZaNw$
EQJMgzaveDDIoiMl&sMHkzdRptRCPVjwW.RSVMjs-bgRkzrBTEa!!oP fRSxq.PLboTMkX'DUYepxIBFAYuxKXe.jeh
sa!3MGFrSjuM:wX!?BTMl!.?,M:bQzPHpYfN!Cbo'MmtDxBkDD3SBjyFdmY'DOqkWeRjlxyJB-bVbfd&jK$-A-w!P InZOmufBNa!oUAWa!wa!Ucn!
dgqUxJ;I&uSqE&oT:zYCJjHNxyjFa-vSPIkqpf iNY.IF$gH'xqtRteh$J;aXlcxz


In [1]:
import torch
import torch.nn as nn 
from torch.nn import functional as F 
torch.manual_seed(1337)
device='cuda' if torch.cuda.is_available() else 'cpu'

In [14]:
with open('input.txt','r',encoding='utf-8') as f:
    text=f.read()
chars=sorted(list(set(text)))
vocab_size=len(chars)
itos={i:ch for i,ch in enumerate(chars)}
stoi={ch:i for i,ch in itos.items()}
encode=lambda s:[stoi[c] for c in s]
decode=lambda l:''.join(itos[i] for i in l)

data=encode(text)
data=torch.tensor(data,dtype=torch.long)
n=int(0.9*len(data))
train=data[:n]
test=data[n:]




block_size=8

def get_batch(split):
    data=train if split=="train" else test 
    ix=torch.randint(len(data)-block_size,(batch_size,))
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1]for i in ix])
    x,y=x.to(device),y.to(device)
    return x,y




class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)
    
    def forward(self,idx,targets=None):
        logits=self.token_embedding_table(idx)
        if targets is None:
            loss=None 
        else:
            B,T,C=logits.shape
            logits=logits.view(B*T,C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits,targets)

        return logits,loss 
    
    def generate(self,idx,max_new_token):
        for _ in range(max_new_token):
            logits,loss=self(idx)
            logits=logits[:,-1,:]
            probs=F.softmax(logits,dim=-1)
            idx.next=torch.multinomial(probs,num_samples=1)
            idx=torch.cat((idx,idx.next),dim=1)
        return idx
    
model=BigramLanguageModel(vocab_size)
m=model.to(device)
logits,loss=m(xb,yb)
optimizer=torch.optim.AdamW(m.parameters(),lr=1e-3)
batch_size=32
for steps in range(10000):
    xb,yb=get_batch("train")

    logits,loss=m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())
context=torch.zeros((1,1),dtype=torch.long,device=device)
print(decode(m.generate(context,max_new_token=500)[0].tolist()))

2.394822597503662

Ke sonidin y y a!
Caril.n y, denjxhece w illd CHAL, mer thoun s's:Conchuntilalllevise sthat dy hangilyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scace, tridesar, wnl'shenou


In [17]:
tri=torch.tril(torch.ones(3,3))
tri=tri/torch.sum(tri,1,keepdim=True)
tri

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

In [18]:
xb.shape

torch.Size([4, 8])

In [3]:
torch.cuda.is_available()

True

In [4]:
yb.shape

NameError: name 'yb' is not defined

In [1]:
import pickle

In [9]:
# import torch
# import torch.nn as nn 
# from torch.nn import functional as F 
# torch.manual_seed(1337)
# device='cuda' if torch.cuda.is_available() else 'cpu'


with open('input.txt','r',encoding='utf-8') as f:
    text=f.read()
chars=sorted(list(set(text)))
vocab_size=len(chars)
itos={i:ch for i,ch in enumerate(chars)}
stoi={ch:i for i,ch in itos.items()}
encode=lambda s:[stoi[c] for c in s]
decode=lambda l:''.join(itos[i] for i in l)

data=encode(text)
data=torch.tensor(data,dtype=torch.long)
n=int(0.9*len(data))
train=data[:n]
test=data[n:]

iters=5000
batch_size=64
block_size=128
lr=3e-4
n_embd=288
n_head=6
n_layer=6
dropout=0.2

def get_batch(split):
    data=train if split=="train" else test 
    ix=torch.randint(len(data)-block_size,(batch_size,))
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1]for i in ix])
    x,y=x.to(device),y.to(device)
    return x,y


@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ["train","test"]:
        losses=torch.zeros(1000)
        for k in range(1000):
            X,Y=get_batch(split)
            logits,loss=model(X,Y)
            losses[k]=loss.item()
        out[split]=losses.mean()
    model.train()
    return out


class Head(nn.Module):
    def __init__(self,head_size):
        super().__init__()
        self.key=nn.Linear(n_embd,head_size,bias=False)
        self.query=nn.Linear(n_embd,head_size,bias=False)
        self.value=nn.Linear(n_embd,head_size,bias=False)
        self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))

        self.dropout=nn.Dropout(dropout)

    def forward(self,x):
        B,T,C=x.shape
        k=self.key(x)
        q=self.query(x)
        v=self.value(x)
        wei=q@k.transpose(-2,-1) / C**0.5
        wei=wei.masked_fill(self.tril[:T,:T]==0,float('-inf'))
        wei=F.softmax(wei,dim=-1)
        wei=self.dropout(wei)
        out=wei@v 
        return out 
    
class MultiHeadAttention(nn.Module):
    def __init__(self,num_heads,head_size):
        super().__init__()
        self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj=nn.Linear(n_embd,n_embd)
        self.dropout=nn.Dropout(dropout)
    def forward(self,x):
        out=torch.cat([h(x) for h in self.heads],dim=-1)
        out=self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self,n_embd):
        super().__init__()
        self.net=nn.Sequential(
            nn.Linear(n_embd,4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd,n_embd),
            nn.Dropout(dropout)
        )
    def forward(self,x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self,n_embd,n_head):
        super().__init__()
        head_size=n_embd//n_head
        self.sa=MultiHeadAttention(n_head,head_size)
        self.ffwd=FeedForward(n_embd)
        self.ln1=nn.LayerNorm(n_embd)
        self.ln2=nn.LayerNorm(n_embd)
    def forward(self,x):
        x=x+self.sa(self.ln1(x))
        x=x+self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,n_embd)
        self.pos_embedding_table=nn.Embedding(block_size,n_embd)
        self.blocks=nn.Sequential(*[Block(n_embd,n_head=n_head) for _ in range(n_layer)])
        self.ln=nn.LayerNorm(n_embd)
        self.lm_head=nn.Linear(n_embd,vocab_size)
    def forward(self,idx,targets=None):
        B,T=idx.shape
        tok_emb=self.token_embedding_table(idx)
        pos_emb=self.pos_embedding_table(torch.arange(T,device=device))
        x=tok_emb+pos_emb 
        x=self.blocks(x)
        logits=self.lm_head(x)

        if targets is None:
            loss=None 
        else:
            B,T,C=logits.shape
            logits=logits.view(B*T,C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits,targets)
        return logits,loss 
    
    def generate(self,idx,max_new_token):
        for _ in range(max_new_token):
            idx_cond=idx[:,-block_size:]
            logits,loss=self(idx_cond)
            logits=logits[:,-1,:]
            probs=F.softmax(logits,dim=-1)
            idx.next=torch.multinomial(probs,num_samples=1)
            idx=torch.cat((idx,idx.next),dim=1)
        return idx

In [19]:
pickled_model=pickle.load(open("gpt8k.pkl","rb"))

In [53]:
# context=torch.zeros((1,1),dtype=torch.long,device=device)
print(decode(pickled_model.generate(context,max_new_token=1000)[0].tolist()))

BRUTUS:
My lord.

KING RICHARD III:
Your lord some way, and hear it.

JULIET:
O eleven my love, that have I waste sord stain,
And, at only a thousand  distorment.
Thou hadst not enforce putt his tears of his:
Wither repute thy brother's liken to be maid,
Past wiell.

CAMILLIUS:
I am with tears to contempt.

MENTIO:
Show holds with me broke: but that I should
not like make pabella.

TRANIO:
Who's good priests,
Some let once take time my tongue's worsed Cepulis.
Go RIVERRY:
Who is in the common pupet horse 'stands,'
To utter not the enterpite use that not come of truth,
But still peace to kiss.
Which was a comfort worser blon
From one, with in citizen an out vantage?
Besect you may read by them and part,
But put me beloved me ashe;
The crows it is my master, therein senseion's majesty.
My holy eyes I changua eld in our auddor.

GLOUCESTER:
Good make young lady marnify mann gone.

KING EDWARD IV:
Art take in arrand them coronar's sake, to you;
Thy bird is it, and does most fllows,
And pat

In [52]:
context=encode("BRUTUS:")
context=(torch.tensor(context,dtype=torch.long,device=device))
context=context.view(1,-1)
decode(context[0].tolist())

'BRUTUS:'